In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableParallel
load_dotenv()

True

In [2]:
model = ChatOpenAI(model="gpt-4o")

In [3]:
summary_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic"),
        ("human", "Provide a brief summary of the movie {movie_name}"),
    ]
)

In [4]:
plot_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic"),
        ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?")
    ]
)

In [12]:
character_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie critic"),
    ("human", "Analyze the characters: {character}. What are their strengths and weaknesses?"),
])

In [6]:
plot_branch_chain = RunnableLambda(lambda x: plot_template.format_prompt(plot=x)) | model | StrOutputParser()

In [13]:
character_branch_chain = RunnableLambda(lambda x: character_template.format_prompt(character=x)) | model | StrOutputParser()

In [8]:
def combine_analysis(plot_analysis, character_analysis):
    return f"Plot Analysis: \n{plot_analysis}\n\nCharacter Analysis: \n{character_analysis}\n"

In [14]:
chain = (
    summary_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={
        "plot": plot_branch_chain,
        "character": character_branch_chain
    })
    | RunnableLambda(lambda x: combine_analysis(x["branches"]["plot"], x["branches"]["character"]))
)

In [15]:
reslt = chain.invoke({"movie_name": "Iron Man"})

In [16]:
print(reslt)

Plot Analysis: 
"Iron Man," as the inaugural film of the Marvel Cinematic Universe (MCU), possesses several notable strengths and a few weaknesses, both of which contribute to its enduring legacy.

**Strengths:**

1. **Robert Downey Jr.'s Performance:** One of the greatest strengths of "Iron Man" is undoubtedly Robert Downey Jr.'s portrayal of Tony Stark. His charisma, wit, and depth breathe life into a character that could easily have come across as superficial. Downey's ability to balance Stark's arrogance with vulnerability makes the character relatable and endearing.

2. **Character Development:** The film does an excellent job of developing Tony Stark's character arc. He transitions from a carefree playboy primarily interested in profit to a responsible hero aware of the ramifications of his actions. This evolution is convincingly portrayed and is central to the film's narrative strength.

3. **Special Effects and Action Sequences:** The film's visual effects are top-notch, especi